# Inizializzazione pySpark
Tramite il package `findspark` inizializzo `SparkContext`

In [ ]:
import findspark

findspark.init();

import pyspark

context = pyspark.SparkContext(appName="LID Project");

In [4]:
import sys

sys.path.append("..")

from utils.read import read_fvecs

# PCA
## Caricamento del Dataset
Proviamo in primis a caricare il Dataset, una matrice 10000x128 di `np.float32`

In [6]:
dataset = read_fvecs("data/siftsmall_base.fvecs")

## Creazione RDD
Ora tramite `parallelize()` creiamo l'RDD che conterrà la matrice

In [ ]:
rdd = context.parallelize(dataset)

## Dati per PCA
Per procedere all'esecuzione di PCA ci servono dei dati di partenza quali:
* N: Dimensione del dataset
* dim: Dimensionalità del dataset
* mean: Media del dataset
* cov: Matrice di covarianza

In [ ]:
import numpy as np

mean = np.mean(rdd.collect(), axis=0)
cov = np.cov(rdd.collect())

## Autovettori e Autovalori
Calcolo degli autovettori e autovalori a partire da `cov` e sorting

In [ ]:
eigval, eigvec = np.linalg.eig(cov)

## Plotting
Come è possibile vedere: 
* Con 12 PC abbiamo rappresentatività per (circa) il 70% del dataset 
* Con 6 (circa) il 60%
* Con 3 tra il 50% ed il 60%

In [ ]:
tot = sum(eigval)

var_exp = [(i/tot) for i in sorted(eigval, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

import matplotlib.pyplot as plt

plt.step(range(1, 129),cum_var_exp[:128], where='mid')

plt.xlabel('Indice PC')
plt.ylabel('Coverage')

plt.tight_layout()
plt.show()

# Sorting

In [ ]:
sorted_eigvals = sorted(eigval, reverse=True)